In [1]:
import sys

sys.path.append("..")
import numpy as np
import pandas as pd
import pyomo.environ as pyo
import tensorflow.keras as keras
from omlt import OffsetScaling, OmltBlock
from omlt.io.keras import load_keras_sequential
from omlt.neuralnet import (
    FullSpaceNNFormulation,
    FullSpaceSmoothNNFormulation,
    NetworkDefinition,
    ReducedSpaceSmoothNNFormulation,
    ReluBigMFormulation,
    ReluComplementarityFormulation,
    ReluPartitionFormulation,
)
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD, Adam, RMSprop

from src.neural_network import data_scaling



'_BlockData'. The class '_BlockData' has been renamed to 'BlockData'.
(deprecated in 6.7.2) (called from
c:\Users\mexis\anaconda3\envs\main\lib\site-packages\omlt\block.py:33)


In [2]:
model = pyo.ConcreteModel()
model.x = pyo.Var([1, 2], domain=pyo.NonNegativeReals)
model.OBJ = pyo.Objective(expr=2 * model.x[1] + 3 * model.x[2])
model.Constraint1 = pyo.Constraint(expr=3 * model.x[1] + 4 * model.x[2] >= 1)

solver = pyo.SolverFactory("cplex", sense=pyo.minimize)
solver.solve(model)

print("x[1] =", pyo.value(model.x[1]))
print("x[2] =", pyo.value(model.x[2]))
print("OBJ =", pyo.value(model.OBJ))

x[1] = 0.3333333333333333
x[2] = 0.0
OBJ = 0.6666666666666666


In [3]:
# Generate dataset for OMLT
df = pd.DataFrame(columns=["x1", "x2", "OBJ"])

inputs = ['x1', 'x2']
outputs = ['OBJ']

for i in range(1000):
    x1 = np.random.rand()
    x2 = np.random.rand()
    OBJ = 2 * x1 + 3 * x2
    df.loc[i] = [x1, x2, OBJ]

df.head()

,x1,x2,OBJ
0,0.587367,0.789526,3.543310
1,0.403378,0.552026,2.462834
2,0.237654,0.372139,1.591724
3,0.713976,0.283278,2.277787
4,0.300572,0.177278,1.132979


In [4]:
# Data scaling
x, y, x_offset, x_factor, y_offset,\
    y_factor, scaled_lb, scaled_ub = data_scaling(df, inputs, outputs)


In [5]:
# ANN
model = Sequential()
model.add(Dense(32, input_dim=2, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss="mean_squared_error", optimizer=Adam(learning_rate=0.0005))
history = model.fit(
    x, y, epochs=500, batch_size=64, validation_split=0.33, verbose=2
)
model.save('simple_model.keras')


Epoch 1/500

11/11 - 1s - loss: 1.0386 - val_loss: 0.9217 - 1s/epoch - 127ms/step
Epoch 2/500
11/11 - 0s - loss: 0.8865 - val_loss: 0.7821 - 95ms/epoch - 9ms/step
Epoch 3/500
11/11 - 0s - loss: 0.7545 - val_loss: 0.6655 - 80ms/epoch - 7ms/step
Epoch 4/500
11/11 - 0s - loss: 0.6412 - val_loss: 0.5622 - 119ms/epoch - 11ms/step
Epoch 5/500
11/11 - 0s - loss: 0.5394 - val_loss: 0.4624 - 104ms/epoch - 9ms/step
Epoch 6/500
11/11 - 0s - loss: 0.4385 - val_loss: 0.3685 - 97ms/epoch - 9ms/step
Epoch 7/500
11/11 - 0s - loss: 0.3452 - val_loss: 0.2813 - 97ms/epoch - 9ms/step
Epoch 8/500
11/11 - 0s - loss: 0.2585 - val_loss: 0.2043 - 95ms/epoch - 9ms/step
Epoch 9/500
11/11 - 0s - loss: 0.1845 - val_loss: 0.1389 - 94ms/epoch - 9ms/step
Epoch 10/500
11/11 - 0s - loss: 0.1215 - val_loss: 0.0898 - 111ms/epoch - 10ms/step
Epoch 11/500
11/11 - 0s - loss: 0.0770 - val_loss: 0.0550 - 150ms/epoch - 14ms/step
Epoch 12/500
11/11 - 0s - loss: 0.0469 - val_loss: 0.0328 - 75ms/epoch - 7ms/step
Epoch 13/500
11/

In [6]:
omlt_model = pyo.ConcreteModel()
omlt_model.nn = OmltBlock()
nn = keras.models.load_model('simple_model.keras')

scaler = OffsetScaling(
    offset_inputs={i: x_offset[inputs[i]] for i in range(len(inputs))},
    factor_inputs={i: x_factor[inputs[i]] for i in range(len(inputs))},
    offset_outputs={i: y_offset[outputs[i]] for i in range(len(outputs))},
    factor_outputs={i: y_factor[outputs[i]] for i in range(len(outputs))},
)

scaled_input_bounds = {i: (scaled_lb[i], scaled_ub[i]) for i in range(len(inputs))}

net = load_keras_sequential(
    nn, scaling_object=scaler, scaled_input_bounds=scaled_input_bounds
)

# Which formulation to use?
omlt_model.nn.build_formulation(ReluComplementarityFormulation(net))
    
# FullSpaceNNFormulation, 
# FullSpaceSmoothNNFormulation,
# NetworkDefinition,
# ReducedSpaceSmoothNNFormulation,
# ReluBigMFormulation,
# ReluComplementarityFormulation,
# ReluPartitionFormulation,


In [7]:
omlt_model.obj = pyo.Objective(expr=omlt_model.nn.outputs[0], sense=pyo.minimize)

omlt_model.Constraint1 = pyo.Constraint(expr=3 * omlt_model.nn.inputs[0] + 4 * omlt_model.nn.inputs[1] >= 1)


solver = pyo.SolverFactory("ipopt")
solver.solve(omlt_model)

print("x[1] =", pyo.value(omlt_model.nn.inputs[0]))
print("x[2] =", pyo.value(omlt_model.nn.inputs[1]))
print("OBJ =", pyo.value(omlt_model.obj))

x[1] = 0.33332421310017185
x[2] = 6.838616914129086e-06
OBJ = 0.6682969034808739
